<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema5_6_padding_truncation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# 5.6. Procesos de padding y truncation en transformers

En un ejercicio anterior, aprendimos a tokenizar un texto con un tokenizador de un modelo transformer.

En este ejercicio, vamos practicaremos con los procesos de padding y truncation, que nos van a ayudar a que todas las entradas del transformer tengan el mismo tamaño.





## Instalación librería transformers

El primer paso será instalar la librería transformers de Hugging Face que nos permitirá cargar el modelo tranformer y su tokenizador. Además, vamos a instalar la librería datasets.
Las instalaremos en modo silencioso (-q) para que no muestra las dependencias durante la instalación.


In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.5 MB/s eta 0:00:00


## Cargar tokenizador

Para cargar el tokenizador, usaremos la clase **AutoTokenizer**, a la que simplemente tendremos que indicarle en su método **from_pretrained** el tokenizador del modelo tranformer que queremos utilizar. En este ejercicio, usaremos BERT Base en su versión uncased


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

## Cargar un dataset
También vamos a cargar un dataset (split train de 'rottten_tomamotes'). Puedes cargar todo el split, aunque para poder ver mejor los resultados te recomendamos que únicamente tomes 10 registros.

In [ ]:
from datasets import load_dataset
# data = load_dataset("rotten_tomatoes", split='train')
data = load_dataset("rotten_tomatoes", split='train[0:10]')
data

Dataset({
    features: ['text', 'label'],
    num_rows: 10
})

## Tokenizar la colección de textos

In [ ]:
encoded_inputs = tokenizer(data['text'])
encoded_inputs

{'input_ids': [[101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102], [101, 1996, 9882, 2135, 9603, 13633, 1997, 1000, 1996, 2935, 1997, 1996, 7635, 1000, 11544, 2003, 2061, 4121, 2008, 1037, 5930, 1997, 2616, 3685, 23613, 6235, 2522, 1011, 3213, 1013, 2472, 2848, 4027, 1005, 1055, 4423, 4432, 1997, 1046, 1012, 1054, 1012, 1054, 1012, 23602, 1005, 1055, 2690, 1011, 3011, 1012, 102], [101, 4621, 2021, 2205, 1011, 8915, 23267, 16012, 24330, 102], [101, 2065, 2017, 2823, 2066, 2000, 2175, 2000, 1996, 5691, 2000, 2031, 4569, 1010, 2001, 28518, 2003, 1037, 2204, 2173, 2000, 2707, 1012, 102], [101, 19391, 2004, 2242, 4678, 1010, 2019, 3277, 3185, 2008, 1005, 1055, 2061, 7481, 1998, 10326, 2135, 5159, 2008, 2009, 2987, 1005, 1056, 2514, 2066, 2028, 1012, 102], [

Podemos ver que sigue devolviendo un diccionario con los tres campos input_ids, token_type_ids y attention_mask, pero ahora cada campo no contiene una lista, sino contiene una lista de sublistas, donde cada sublista es uno de los textos en la colección de entrada.

Podemos ver la codificación del primer texto:

In [ ]:
print(data['text'][0])
print("input_ids:", encoded_inputs.input_ids[0])
print("token_type_ids:", encoded_inputs.token_type_ids[0])
print("attention_mask:", encoded_inputs.attention_mask[0])


the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
input_ids: [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Vamos a decodificar los input_ids, parpa ver si el texto coincide con el primer texto (vemos que en efecto coinciden):

In [ ]:
tokenizer.convert_ids_to_tokens(encoded_inputs.input_ids[0])


['[CLS]',
 'the',
 'rock',
 'is',
 'destined',
 'to',
 'be',
 'the',
 '21st',
 'century',
 "'",
 's',
 'new',
 '"',
 'conan',
 '"',
 'and',
 'that',
 'he',
 "'",
 's',
 'going',
 'to',
 'make',
 'a',
 'splash',
 'even',
 'greater',
 'than',
 'arnold',
 'schwarz',
 '##ene',
 '##gger',
 ',',
 'jean',
 '-',
 'cl',
 '##aud',
 'van',
 'dam',
 '##me',
 'or',
 'steven',
 'sega',
 '##l',
 '.',
 '[SEP]']

Vamos a ver la longitud de las listas. Podemos ver que las tres listas que contiene encoded_inputs, tienen la misma longitud que data, es decir, que el número de textos que hemos tokenizado.

In [ ]:
print("número de textos:", len(data))
print("número de vectores en input_ids:", len(encoded_inputs.input_ids))
print("número de vectores en token_type_ids:", len(encoded_inputs.token_type_ids))
print("número de vectores en attention_mask:", len(encoded_inputs.attention_mask))

número de textos: 10
número de vectores en input_ids: 10
número de vectores en token_type_ids: 10
número de vectores en attention_mask: 10


Ahora vamos a ver el tamaño de cada una de las sublistas. Será suficiente con estudiar el tamaño de las sublistas en input_ids:

In [ ]:
sizes=[]

for i in range(len(encoded_inputs.input_ids)):
    print('Texto: ', data[i])
    print('número de tokens en el texto', i, " : ", len(encoded_inputs.input_ids[i]))
    sizes.append(len(encoded_inputs.input_ids[i]))


Texto:  {'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
número de tokens en el texto 0  :  47
Texto:  {'text': 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'label': 1}
número de tokens en el texto 1  :  52
Texto:  {'text': 'effective but too-tepid biopic', 'label': 1}
número de tokens en el texto 2  :  10
Texto:  {'text': 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .', 'label': 1}
número de tokens en el texto 3  :  24
Texto:  {'text': "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .", 'label': 1}
número de tokens en el texto 4  :  28
Texto:  

Podemos ver que cada sublista (cada texto) tiene un tamaño distinto (número de tokens distinto). Esto es completamente normal en cualquier colección de textos. Sin embargo, las redes neuronales, y en particular los transformers, necesitan que todas sus entradas, tengan el mismo tamaño.

## Padding y truncation
¿Cómo podemos conseguir qué todos los textos tengan el mismo tamaño (número de tokens)?. En este curso, ya hemos hablado de estos dos procesos que son muy comunes en redes neuronales para conseguir que todas las entradas a una red tengan el mismo tamaño. Vamos a recordarlas brevemente:

- **Truncation** consiste en eliminar tokens de un texto.
- **Padding** consiste en añadir un token especial (índice 0).




En primer lugar, deberemos estimar un tamaño que sea lo suficientemente representativo de toda la colección.

In [ ]:
print('Tamaños de los textos:', sizes)
print()

import pandas as pd
s = pd.Series(sizes)
print('Estadística de la muestra')
s.describe()


Tamaños de los textos: [47, 52, 10, 24, 28, 32, 11, 22, 34, 17]

Estadística de la muestra


count    10.000000
mean     27.700000
std      14.040022
min      10.000000
25%      18.250000
50%      26.000000
75%      33.500000
max      52.000000
dtype: float64

Vemos que el tamaño medio es de 27.7 tokens y el tamaño máximo 52, y su percentil 75 es 33.5.
Si tenemos una colección muy grande de textos y sus tamaños son muy diferentes, seleccionar el tamaño máximo podría ser una opción que nos haga desperdiciar memoria.

En este caso una buena elección, podría ser 34 tokens, pero vamos a seleccionar únicamente 20 tokens, porque esto nos va a permitir ver de forma más sencilla el efecto del padding y truncation.


Ya podemos aplicar el tokenizador, indicando ahora el tamaño máximo de nuestras entradas (hemos seleccionado 20), y también qué procesos vamos a utilizar. Vamos a ver su salida

In [ ]:
batch = tokenizer(data['text'], max_length=20, padding=True, truncation=True, return_tensors="pt")

In [ ]:
batch['input_ids']

tensor([[  101,  1996,  2600,  2003, 16036,  2000,  2022,  1996,  7398,  2301,
          1005,  1055,  2047,  1000, 16608,  1000,  1998,  2008,  2002,   102],
        [  101,  1996,  9882,  2135,  9603, 13633,  1997,  1000,  1996,  2935,
          1997,  1996,  7635,  1000, 11544,  2003,  2061,  4121,  2008,   102],
        [  101,  4621,  2021,  2205,  1011,  8915, 23267, 16012, 24330,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2065,  2017,  2823,  2066,  2000,  2175,  2000,  1996,  5691,
          2000,  2031,  4569,  1010,  2001, 28518,  2003,  1037,  2204,   102],
        [  101, 19391,  2004,  2242,  4678,  1010,  2019,  3277,  3185,  2008,
          1005,  1055,  2061,  7481,  1998, 10326,  2135,  5159,  2008,   102],
        [  101,  1996,  2143,  3640,  2070,  2307, 12369,  2046,  1996, 11265,
         10976,  4588,  9273,  3388,  1997,  2035,  5888,  1011,  1011,   102],
        [  101,  4107,  2008,  4678,  5257,  1

Podemos ver que ahora sí todas las secuencias tienen la misma longitud:

In [ ]:
for sublist in batch['input_ids']:
    print(len(sublist))

20
20
20
20
20
20
20
20
20
20


En el primer texto, podemos que se han eliminado tokens del final:

In [ ]:
print(data[0])

print('input_ids  después de padding y truncation ', batch.input_ids[0])
print('número de tokens después de padding y truncation', len(batch.input_ids[0]))

print()
print('input_ids  sin de padding y truncation ', encoded_inputs.input_ids[0])
print('número de tokens sin padding y truncation', len(encoded_inputs.input_ids[0]))



{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
input_ids  después de padding y truncation  tensor([  101,  1996,  2600,  2003, 16036,  2000,  2022,  1996,  7398,  2301,
         1005,  1055,  2047,  1000, 16608,  1000,  1998,  2008,  2002,   102])
número de tokens después de padding y truncation 20

input_ids  sin de padding y truncation  [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102]
número de tokens sin padding y truncation 47


La listas de attention_mask cambian de tamaño, pero para el primer texto, sus valores son siempre 1.

In [ ]:
print('attention_mask después de padding y truncation: ', batch.attention_mask[0])
print('attention_mask sin padding y truncation: ',  encoded_inputs.attention_mask[0])


attention_mask después de padding y truncation:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
attention_mask sin padding y truncation:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Sin embargo, en siguiente texto sí se ha aplicado padding:

In [ ]:
print(data[6])
print('input_ids  después de padding y truncation ', batch.input_ids[6])
print('número de tokens después de padding y truncation', len(batch.input_ids[6]))

print()
print('input_ids  sin de padding y truncation ', encoded_inputs.input_ids[6])
print('número de tokens sin padding y truncation', len(encoded_inputs.input_ids[6]))



{'text': 'offers that rare combination of entertainment and education .', 'label': 1}
input_ids  después de padding y truncation  tensor([ 101, 4107, 2008, 4678, 5257, 1997, 4024, 1998, 2495, 1012,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
número de tokens después de padding y truncation 20

input_ids  sin de padding y truncation  [101, 4107, 2008, 4678, 5257, 1997, 4024, 1998, 2495, 1012, 102]
número de tokens sin padding y truncation 11


Vemos que ahora sí las listas de attention_mask que devuelve el tokenizador (aplicando padding y truncation) sí contiene ya 0's (el padding se aplica al final del texto)

In [ ]:
print('attention_mask sin padding y truncation: ',  encoded_inputs.attention_mask[6])
print('attention_mask después de padding y truncation: ', batch.attention_mask[6])


attention_mask sin padding y truncation:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask después de padding y truncation:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])


¿Qué ocurre si no específicamos un tamaño máximo?

In [ ]:
batch = tokenizer(data['text'], padding=True, return_tensors="pt")


El tokenizador tomará como valor máixmo el mayor tamaño (52), y en este caso, el argumento de truncation no tendrá ningún efecto, ya que todas las entradas tendrán la longitud máxima.

In [ ]:
print(len(batch.input_ids[1]), batch.input_ids[1])
print(len(batch.input_ids[6]), batch.input_ids[6])


52 tensor([  101,  1996,  9882,  2135,  9603, 13633,  1997,  1000,  1996,  2935,
         1997,  1996,  7635,  1000, 11544,  2003,  2061,  4121,  2008,  1037,
         5930,  1997,  2616,  3685, 23613,  6235,  2522,  1011,  3213,  1013,
         2472,  2848,  4027,  1005,  1055,  4423,  4432,  1997,  1046,  1012,
         1054,  1012,  1054,  1012, 23602,  1005,  1055,  2690,  1011,  3011,
         1012,   102])
52 tensor([ 101, 4107, 2008, 4678, 5257, 1997, 4024, 1998, 2495, 1012,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])


Vamos a revisar los distintos argumentos de la tokenización:


- **max_length**: controla la longitud para las operaciones de padding y truncation. Si no se usa este parámetro, su valor será la longitud máxima que el modelo puede aceptar (normalmente 512 tokens).
- **padding**: su valor puede ser:
    - True o 'longest' añaden padding considerando la oración más larga en el batch.
    - 'max_length' añade padding considerando la longitud máxima indicada en el argumento **max_length**.
    - False o 'do_not_pad', no se aplica padding.
    
- **Truncation**, su valor puede ser:
    - True o 'longest_first' trunca las oraciones a la longitud máxima que se ha especificado **max_length** o la máxima longitud permitida por el modelo (por ejemplo, 512 tokens en el caso de la mayoría de los modelos). Si hay dos oraciones, se trunca token por token en la oración más larga del par, hasta alcanzar la longitud adecuada.
    
    - 'only_second' trunca las oraciones a la longitud máxima que se ha especificado **max_length** o la máxima longitud permitida por el modelo. Solo truncará la segunda oración de un par si se proporciona un par de secuencias (o un lote de pares de secuencias).

    - 'only_first'  trunca las oraciones a la longitud máxima que se ha especificado **max_length** o la máxima longitud permitida por el modelo. Solo truncará la primera oración de un par si se proporciona un par de secuencias (o un lote de pares de secuencias).

    - False 0 'do_not_truncate' no trunca la oración.


Te recomiendo que visites el siguiente enlace https://huggingface.co/docs/transformers/pad_truncation donde encontrarás más información sobre los distintos valores de los argumentos **padding** y **truncation** que puedes utilizar para procesar una colección de textos.